In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import networkx as nx
import json
import sys
#import seaborn as sns
#sns.set()
#%matplotlib inline

## Step1: build the initial state of the entire user network, as well as the purchae history of the users
Input: sample_dataset/batch_log.json

In [2]:
batchlogfile = 'sample_dataset/batch_log_s.json'
df = pd.read_json(batchlogfile, lines=True)

In [3]:
df.head(10)

,D,T,amount,event_type,id,id1,id2,timestamp
0,1.0,50.0,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,NaN,20.68,purchase,2.0,NaN,NaN,2017-06-14 18:46:40
2,NaN,NaN,27.91,purchase,3.0,NaN,NaN,2017-06-14 18:46:40
3,NaN,NaN,76.61,purchase,4.0,NaN,NaN,2017-06-14 18:46:40
4,NaN,NaN,56.01,purchase,5.0,NaN,NaN,2017-06-14 18:46:40
5,NaN,NaN,73.72,purchase,6.0,NaN,NaN,2017-06-14 18:46:40
6,NaN,NaN,24.11,purchase,7.0,NaN,NaN,2017-06-14 18:46:40
7,NaN,NaN,33.43,purchase,8.0,NaN,NaN,2017-06-14 18:46:40
8,NaN,NaN,49.89,purchase,9.0,NaN,NaN,2017-06-14 18:46:40
9,NaN,NaN,24.25,purchase,10.0,NaN,NaN,2017-06-14 18:46:40


In [4]:
df.describe()

,D,T,amount,id,id1,id2
count,1.0,1.0,9.000000,9.000000,13.000000,13.000000
mean,1.0,50.0,42.956667,6.000000,4.769231,6.461538
std,NaN,NaN,21.840146,2.738613,2.803386,3.230464
min,1.0,50.0,20.680000,2.000000,1.000000,2.000000
25%,1.0,50.0,24.250000,4.000000,3.000000,3.000000
50%,1.0,50.0,33.430000,6.000000,4.000000,6.000000
75%,1.0,50.0,56.010000,8.000000,7.000000,10.000000
max,1.0,50.0,76.610000,10.000000,9.000000,10.000000


In [5]:
# Read D and T
df_DT=df[df['D'].notnull()]
df_DT=df_DT[['D','T']]
D = df_DT.values[0][0]
T = df_DT.values[0][1]
print(D)
print(T)
df_DT.head()

1.0
50.0


,D,T
0,1.0,50.0


In [6]:
# check D and T values
if D < 1:
    print('Program terminated because of D < 1')
    sys.exit()
if T < 2:
    print('Program terminated because of T < 2')
    sys.exit()

In [7]:
#for possible_value in set(df['event_type'].tolist()):
#    print(possible_value)

In [8]:
df_purchase = df[df['event_type']=='purchase']
df_purchase = df_purchase[['event_type','id','timestamp','amount']]
df_purchase = df_purchase.sort_values('timestamp')
df_purchase.shape

(9, 4)

In [9]:
df_friend=df[(df['event_type']=='befriend') | (df['event_type']=='unfriend')]
df_friend=df_friend[['event_type','id1','id2','timestamp']]
df_friend=df_friend.sort_values('timestamp')
df_friend.shape

(13, 4)

In [10]:
G = nx.Graph()

In [11]:
id1list = df_friend.id1.tolist()
id2list = df_friend.id2.tolist()
idlist = set(id1list + id2list)
G.add_nodes_from(idlist)
len(list(G.nodes()))

10

In [12]:
def Add_edges(data):
    for i in range(0, len(data)):
        datai = data.iloc[i]
        id10 = datai['id1']
        id20 = datai['id2']
        event_type0 = datai['event_type']
        if event_type0 == 'befriend':
            G.add_edge(id10,id20)
        if event_type0 == 'unfriend':
            if G.has_edge(id10,id20):
                G.remove_edge(id10,id20)  
                            

In [13]:
Add_edges(df_friend)

In [14]:
len(list(G.edges()))

10

In [15]:
nnode = 1.0

df_friend[df_friend['id1'] == nnode]

,event_type,id1,id2,timestamp
10,befriend,1.0,2.0,2017-06-14 18:46:40
11,befriend,1.0,3.0,2017-06-14 18:46:40
13,befriend,1.0,8.0,2017-06-14 18:46:40


In [16]:
#G[10.0]

In [17]:
G.number_of_nodes()

10

In [18]:
G.number_of_edges()

10

In [19]:
#list(G.edges())

In [20]:
#list(G.neighbors(nnode))

In [21]:
#list(G.neighbors(3.0))

In [22]:
#nnode = 1.0

Dlist = list(G.neighbors(nnode))
#print(Dlist)
for i in list(G.neighbors(nnode)):
    #print(list(G.neighbors(i)))
    Dlist += list(G.neighbors(i))
    #print(Dlist)
    #print(len(list(G.neighbors(i))))

#print(Dlist)
#print(set(Dlist))
print(len(set(Dlist)))



7


In [23]:
len(list(nx.ego_graph(G, nnode, D, center=False)))

3

In [24]:
# define a dataframe: df_mean_sd to store the mean and sd of the last T purchase for each user's network
#df_mean_sd = pd.DataFrame({"id": [], "mean": [], "sd": []})
#df_mean_sd

In [25]:
# define a function to calcualte the mean and sd for userid's network
def Get_Mean_SD(userid):
    Nodes = list(nx.ego_graph(G, userid, D, center=False))
    df_Nodes = df_purchase.loc[df_purchase['id'].isin(Nodes)]
    if len(df_Nodes) >= 2:    
        if len(df_Nodes) > T:
            df_Nodes = df_Nodes.sort_values('timestamp').iloc[-int(T):]
        #df_Nodes.shape
        mean = df_Nodes.amount.mean()
        sd = df_Nodes.amount.std()
        mean = float("{0:.2f}".format(mean))
        sd = float("{0:.2f}".format(sd))
    #Node_amount = pd.DataFrame({'id': [userid], 'mean': [mean], 'sd': [sd]})
    #global df_mean_sd
    #df_mean_sd = df_mean_sd.append(Node_amount)
    else:
        mean=np.nan
        sd=np.nan
    
    return mean, sd, len(df_Nodes), df_purchase.shape[0]

In [26]:
#Get_Mean_SD(4.0)

In [27]:
#df_mean_sd

In [28]:
df_purchase

,event_type,id,timestamp,amount
1,purchase,2.0,2017-06-14 18:46:40,20.68
2,purchase,3.0,2017-06-14 18:46:40,27.91
3,purchase,4.0,2017-06-14 18:46:40,76.61
4,purchase,5.0,2017-06-14 18:46:40,56.01
5,purchase,6.0,2017-06-14 18:46:40,73.72
6,purchase,7.0,2017-06-14 18:46:40,24.11
7,purchase,8.0,2017-06-14 18:46:40,33.43
8,purchase,9.0,2017-06-14 18:46:40,49.89
9,purchase,10.0,2017-06-14 18:46:40,24.25


## Step2: Determine where a purchase is anomalous 
input file: sample_dataset/stream_log.json

In [29]:
# read in the stream_log.json
streamlogfile = 'sample_dataset/stream_log_s.json'
df_stream = pd.read_json(streamlogfile, lines=True)
# If sort on the timestamp is needed, commentout the following line
#df_stream = df_stream.sort_values('timestamp')

# open output file flagged_purchases.json
#flaggedfile = 'log_output/flagged_purchases.json'
flaggedfile = 'flagged_purchases_s.json'
f = open(flaggedfile, 'w')

In [30]:
# Determine a purchase is anomalous; update purchase history; update social network
for i in range(0, len(df_stream)):
    datai = df_stream.iloc[i]
    event_type = datai['event_type']
    if event_type == 'purchase':
        # update purchase history
        df_purchase = df_purchase.append(datai, ignore_index=True)
        timestamp = datai['timestamp']
        timestamp = str(timestamp)
        userid = datai['id']
        amount = datai['amount']
        mean, sd, TT, N = Get_Mean_SD(userid)
        print('userid: {0:.1f} mean: {1:.2f} sd: {2:.2f} TT: {3:d} N: {4:d}'.format(userid, mean, sd, TT, N))
        if mean != np.nan:
            mean_3sd = mean + (3*sd)
            dataflag = {"event_type":event_type, "timestamp":str(timestamp), "id": userid, "amount": amount, "mean": mean, "sd": sd}
            if amount > mean_3sd:
                f.write('"event_type":"{0:s}", "timestamp":"{1:s}", "id": "{2:.0f}", "amount": "{3:.2f}", "mean": "{4:.2f}", "sd": "{5:.2f}"\n'.format(event_type, timestamp, userid, amount, mean, sd))
    # update social network
    if event_type == 'befriend':
        id1 = datai['id1']
        id2 = datai['id2']
        G.add_edge(id1,id2)
    if event_type == 'unfriend':
        id1 = datai['id1']
        id2 = datai['id2']
        if G.has_edge(id1,id2):
            G.remove_edge(id1,id2)  
    

userid: 2.0 mean: nan sd: nan TT: 1 N: 10
userid: 1.0 mean: 23.18 sd: 9.83 TT: 4 N: 11
userid: 2.0 mean: 48.29 sd: 2.26 TT: 2 N: 12
userid: 3.0 mean: 48.17 sd: 24.84 TT: 3 N: 13
userid: 3.0 mean: 48.17 sd: 24.84 TT: 3 N: 14


In [31]:
f.close() 

In [32]:
df_purchase

,event_type,id,timestamp,amount
0,purchase,2.0,2017-06-14 18:46:40,20.68
1,purchase,3.0,2017-06-14 18:46:40,27.91
2,purchase,4.0,2017-06-14 18:46:40,76.61
3,purchase,5.0,2017-06-14 18:46:40,56.01
4,purchase,6.0,2017-06-14 18:46:40,73.72
5,purchase,7.0,2017-06-14 18:46:40,24.11
6,purchase,8.0,2017-06-14 18:46:40,33.43
7,purchase,9.0,2017-06-14 18:46:40,49.89
8,purchase,10.0,2017-06-14 18:46:40,24.25
9,purchase,2.0,2017-06-14 18:46:40,10.68


In [33]:
G.number_of_nodes()

10

In [34]:
G.number_of_edges()

10